In [2]:
suppressPackageStartupMessages({
                                repo = "http://cran.us.r-project.org"
                                library(caret)
                                library(ggfortify)
                                library(ggplot2)
                                library(dplyr)
                                library(RSQLite)
                                library(DBI)
                                library(class)
                                library(randomForest)
                                })

In [3]:
df = read.csv('for_model_avgs.csv', )
colnames(df)
head(df,5)

[1] "event_date"  "event_name"  "num_flooded" "WDF2"        "WSF2"       
 [6] "AWDR"        "AWND"        "WGF6"        "ht"          "hht"        
[11] "lt"          "llt"         "r15_td"      "rhr_td"      "td_av"      
[16] "gw_av"       "r15mx"       "rhrmx"       "rd"          "r3d"

event_date,event_name,num_flooded,WDF2,WSF2,AWDR,AWND,WGF6,ht,hht,lt,llt,r15_td,rhr_td,td_av,gw_av,r15mx,rhrmx,rd,r3d
2010-09-30 00:00:00,Nicole-2010-09-30,101,121.66667,25.96667,291.66667,12.10000,12.10000,3.2105,3.6175,1.1860,0.5330,1.15693750,0.9268542,1.4529743,4.340699,0.605,1.32250,7.732500,3.518
2010-12-16 00:00:00,Snow-2010-12-16,2,205.00000,15.00000,213.00000,2.70000,2.70000,0.6610,0.8610,-1.1300,-1.2235,-0.03919048,0.1326667,-0.2021396,2.208326,0.030,0.07000,0.227500,0.045
2011-08-27 00:00:00,Irene-2011-08-27,110,158.75000,27.12500,133.30569,12.42137,13.95395,2.2375,4.9680,2.6985,-0.6925,3.20487500,3.1586667,1.0957526,2.334049,0.350,1.10125,6.294167,0.308
2012-10-28 00:00:00,Sandy-2012-10-28,105,18.33333,38.31667,113.78621,18.97307,20.06146,5.2280,5.5760,3.2840,1.5810,4.75382292,4.3288021,2.7493437,2.998493,0.140,0.42625,4.805833,0.341
2013-10-09 00:00:00,Heavy Rain-2013-10-09,36,31.66667,32.55000,40.04515,15.99978,15.31507,3.4925,4.2125,2.0830,1.0845,3.83503125,3.9148646,2.2448646,2.141837,0.375,0.80000,3.671667,0.924


In [7]:
run_model = function(model_type, trn_data, trn_in_data, trn_out_data, tst_in_data, tst_out_data, fmla){
  if (model_type == 'poisson'){
    print('normalizing')
    train_col_stds = apply(trn_in_data, 2, sd)
    train_col_means = colMeans(trn_in_data)
  
    train_normalized = t((t(trn_in_data)-train_col_means)/train_col_stds)
    test_normalized = t((t(tst_in_data)-train_col_means)/train_col_stds)
  
    pca = prcomp(train_normalized)
  
    trn_preprocessed = predict(pca, train_normalized)
    tst_preprocessed = predict(pca, test_normalized)
  
    fmla = as.formula(paste(out_col_name, "~", paste(colnames(trn_preprocessed), collapse="+")))
  
    train_data = cbind(as.data.frame(trn_preprocessed), num_flooded = model_data[prt$Resample1, out_col_name])
    trn_in_data = trn_preprocessed
    tst_in_data = tst_preprocessed
    output = glm(fmla, data=train_data, family = poisson)
  }
  else if (model_type == 'rf'){
    output = randomForest(fmla, data=trn_data, importance = TRUE, ntree=100, mtry=16)
    impo = as.data.frame(output$importance)
    impo = impo[,1]
  }

  pred_trn = predict(output, newdata = as.data.frame(trn_in_data), type='response')
  pred_tst = predict(output, newdata = as.data.frame(tst_in_data), type='response')
  
  if (model_type == 'rf'){
       return(list(pred_trn, pred_tst, impo))
  }
  else {
       return(list(pred_trn, pred_tst))
  }
}

remove_cols= function(l, cols){
    return(l[! l %in% cols])
    }

In [9]:
set.seed(5)
df = df[df[,'rd']>0.01,]
cols_to_remove = c('event_name', 'event_date', 'num_flooded')
in_col_names = remove_cols(colnames(df), cols_to_remove)
out_col_name = 'num_flooded'
model_data = df[, append(in_col_names, out_col_name)]
model_data = na.omit(model_data)
import_df = data.frame(matrix(nrow=length(in_col_names)))
all_pred_tst = c()
all_pred_trn = c()
all_tst = c()
all_trn = c()
fomla = as.formula(paste(out_col_name, "~", paste(in_col_names, collapse="+")))
model_types = c('rf', 'poisson')
suffix = 'out'

In [10]:
for (i in 1:101){
  prt = createDataPartition(model_data[, out_col_name], p=0.7)
  train_data = model_data[prt$Resample1,]
  train_in_data = data.frame(train_data[, in_col_names])
  colnames(train_in_data) = in_col_names
  train_out_data = train_data[, out_col_name]
  test_in_data = data.frame(model_data[-prt$Resample1, in_col_names])
  colnames(test_in_data) = in_col_names
  test_out_data = model_data[-prt$Resample1, out_col_name]
  
  for (model in model_types){
      print(paste("run: ", i, sep = ''))
      model_results = run_model(model, train_data, train_in_data, train_out_data, test_in_data, test_out_data, fomla)
      pred_train = model_results[1]
	  pred_test = model_results[2]

	  all_trn_df = data.frame(train_out_data, unlist(pred_train))
	  colnames(all_trn_df) = c('all_trn', 'all_pred_trn')
	  all_tst_df = data.frame(test_out_data, unlist(pred_test))
	  colnames(all_tst_df) = c('all_tst', 'all_pred_tst')
	  write.table(all_trn_df, paste(model, '_', suffix, '_train.csv', sep=""), append=TRUE,  sep=",", col.names = F)
	  write.table(all_tst_df, paste(model, '_', suffix, '_test.csv', sep=""), append=TRUE,  sep=",", col.names = F)

	  if (model == 'rf'){
      impo = model_results[3]
	    import_df = cbind(import_df, impo)
	  }
	}
}

colnames(import_df) = 1:ncol(import_df)
row.names(import_df) = in_col_names
write.csv(import_df, paste('rf_impo_', suffix, sep=""), append=TRUE)

[1] "run: 1"
[1] "run: 1"
[1] "normalizing"
[1] "run: 2"
[1] "run: 2"
[1] "normalizing"
[1] "run: 3"
[1] "run: 3"
[1] "normalizing"
[1] "run: 4"
[1] "run: 4"
[1] "normalizing"
[1] "run: 5"
[1] "run: 5"
[1] "normalizing"
[1] "run: 6"
[1] "run: 6"
[1] "normalizing"
[1] "run: 7"
[1] "run: 7"
[1] "normalizing"
[1] "run: 8"
[1] "run: 8"
[1] "normalizing"
[1] "run: 9"
[1] "run: 9"
[1] "normalizing"
[1] "run: 10"
[1] "run: 10"
[1] "normalizing"
[1] "run: 11"
[1] "run: 11"
[1] "normalizing"
[1] "run: 12"
[1] "run: 12"
[1] "normalizing"
[1] "run: 13"
[1] "run: 13"
[1] "normalizing"
[1] "run: 14"
[1] "run: 14"
[1] "normalizing"
[1] "run: 15"
[1] "run: 15"
[1] "normalizing"
[1] "run: 16"
[1] "run: 16"
[1] "normalizing"
[1] "run: 17"
[1] "run: 17"
[1] "normalizing"
[1] "run: 18"
[1] "run: 18"
[1] "normalizing"
[1] "run: 19"
[1] "run: 19"
[1] "normalizing"
[1] "run: 20"
[1] "run: 20"
[1] "normalizing"
[1] "run: 21"
[1] "run: 21"
[1] "normalizing"
[1] "run: 22"
[1] "run: 22"
[1] "normalizing"
[1] "r

Warning message in write.csv(import_df, paste("rf_impo_", suffix, sep = ""), append = TRUE):
“attempt to set 'append' ignored”